### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
########
# description of three observable trends based on given data set
# 1) At least half of the player made multi. purchases of item
# 2) Every 5 male player there will be 1 female player
# 3) Male player are 6 times more likely to make a purchase in game than a female player
########

In [2]:
# Dependencies and Setup
import pandas as pd
import locale

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [3]:
# purchase_data["SN"].nunique() == len(purchase_data["SN"].unique())
df = purchase_data["SN"].nunique()
df = pd.DataFrame({"Total Players":[df]})
df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
# apply currency format
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
# set float formatter globally
pd.set_option("float_format", locale.currency)

item = purchase_data["Item ID"].nunique()
purchase = purchase_data["Purchase ID"].nunique()
#totalPrice = purchase_data["Price"].map('{:,.2f}%'.format)
totalPrice = round(purchase_data["Price"].sum(), 2)
avgPrice = round(purchase_data["Price"].mean(), 2)

#### Q1 ####
# put all info in data frame to display
df = pd.DataFrame({"Number of Unique Items":item,
                   "Average Price":[avgPrice], 
                   "Number of Purchases":purchase, 
                   "Total Revenue":totalPrice})
df

#### note on: formatters ####
# this formatter apply to StringValueOf(int)
#df = df.to_string(formatters={'Total Revenue':'${:,.2f}'.format})
# this formatter apply to all digits
#df = df.applymap('{:,.2f}%'.format)
#### end note ####

# reset float formatter globally
#pd.reset_option("float_format")

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
totalPlayer = purchase_data["SN"].nunique()
df = purchase_data[["SN","Gender"]].groupby(['Gender']).nunique()

# calculate values for player percentage
ppl = []
for x in df["SN"]:
    ppl.append((x/totalPlayer)*100)

# adding the new "player percentage" column
df['Percentage of Players'] = ppl
# apply formatter
df['Percentage of Players'] = df['Percentage of Players'].map('{:,.2f}%'.format)

# drop 'Gender' col then sort table by 'SN'
df = df.drop(columns=['Gender']).sort_values(by='SN', ascending=False)
df.rename(columns={"SN":"Total Count"})

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
df1 = purchase_data[["SN","Gender","Price"]].groupby(['Gender']).agg('sum')
df2 = purchase_data[["SN","Gender"]].groupby(['Gender']).agg('count')
joined_df = df1.join(df2, how='left')

# "Average Purchase Price" column
avg_df = purchase_data[["SN","Gender","Price"]].groupby(['Gender']).mean()
joined_df['avg'] = avg_df

count = []
# "Avg Total Purchase per Person" column
uniqueCount_df = purchase_data[["SN","Gender"]].groupby(['Gender']).nunique()
for x,y in zip(df1["Price"], uniqueCount_df["SN"]):
    count.append(x/y)
joined_df['new col'] = count

# reorder the dataframe
joined_df = joined_df[["SN", "avg", "Price", "new col"]]
# rename the columns
joined_df = joined_df.rename(columns={"SN":"Purchase Count", 
                          "avg":"Average Purchase Price",
                          "Price":"Total Purchase Value",
                          "new col":"Avg Total Purchase per Person"})
joined_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 45]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#df = purchase_data[["SN","Age"]].groupby(['Age']).count()
df = purchase_data[["Age","SN"]].groupby(['Age']).nunique()

# re-assign new value for 'Age' col
df2 = purchase_data.sort_values(by='Age', ascending=True)
df2 = df2["Age"].unique()
df["Age"] = df2

# use cut() to map values to buckets and add as new col "Age Group"
df["Age Group"] = pd.cut(df["Age"], bins, labels=group_names, include_lowest=True)

#### Q2 ####
# if we want no index by adding new id col
#df = df.groupby('Age Group', as_index=False)["SN"].sum()

# same as above but not adding index col and shift right
df = df.groupby(['Age Group']).sum()
df = df.drop(columns=['Age'])

df["Percentage of Players"] = df["SN"]/len(purchase_data["SN"].unique())*100
df["Percentage of Players"] = df["Percentage of Players"].map('{:,.2f}%'.format)

df = df.rename(columns={"SN":"Total Count"})
df

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 45]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#df = purchase_data[["Price","Age"]].groupby(['Age']).count()
#df = purchase_data[["Age","Price"]].groupby(['Age']).nunique()
df = purchase_data[["Age","Price","Gender"]].groupby(['Age'])['Price'].agg(['count','mean','sum'])

# re-assign new value for 'Age' col for binning later
df2 = purchase_data.sort_values(by='Age', ascending=True)
df2 = df2["Age"].unique()
df["Age"] = df2

# use cut() to map values to buckets and add as new col "Age Group"
df["Age Group"] = pd.cut(df["Age"], bins, labels=group_names, include_lowest=True)

#### Q2 ####
# if we want no index by adding new id col
#df = df.groupby('Age Group', as_index=False)["Price"].sum()
# same as above but not adding index col and shift right
df = df.groupby(['Age Group']).sum()

df["Average Purchase Price"] = df["sum"]/df["count"]
df["Average Purchase Price"] = df["Average Purchase Price"].map('${:,.2f}'.format)

df["mean"] = df["mean"]/2
df["mean"] = df["mean"].map('${:,.2f}'.format)

# reorder the dataframe
df = df[["count", "Average Purchase Price", "sum", "mean"]]
df = df.rename(columns={"count":"Purchase Count", 
                        "sum":"Total Purchase Value",
                        "mean":"Avg Total Purchase per Person"})
df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.97
10-14,28,$2.96,$82.78,$7.34
15-19,136,$3.04,$412.89,$7.59
20-24,365,$3.05,$1114.06,$7.60
25-29,101,$2.90,$293.00,$6.54
30-34,73,$2.93,$214.00,$7.18
35-39,41,$3.60,$147.67,$8.79
40+,13,$2.94,$38.24,$9.18


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
df = purchase_data[["SN","Price"]].groupby('SN')['Price'].agg(['count','mean','sum'])
df = df.sort_values(by='sum', ascending=False)

df = df.rename(columns={"count":"Purchase Count",
                        "mean":"Average Purchase Price", 
                        "sum":"Total Purchase Value"})
df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
df = purchase_data[["Item ID","Item Name", "Price"]].groupby(['Item ID','Item Name'])['Price'].agg(['count','sum'])
df = df.sort_values(by='count', ascending=False)

df["Item Price"] = df["sum"]/df["count"]
df["Item Price"] = df["Item Price"].map('${:,.2f}'.format)

# reorder the dataframe
df = df[["count", "Item Price", "sum"]]
df = df.rename(columns={"count":"Purchase Count", "sum":"Total Purchase Value"})

df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [11]:
df = df.sort_values(by='Total Purchase Value', ascending=False)
df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
